In [2]:
# https://canvas.vu.nl/courses/72644/files/folder/Assignments
import pandas as pd
import numpy as np

In [4]:
# # sofr rate: 
# folder = 'data'
# df_sofr = pd.read_excel(folder + '/SOFR_data.xlsx')
# df_sofr = df_sofr[['Effective Date', 'Rate Type', 'Rate (%)', '1st Percentile (%)',
#        '25th Percentile (%)', '75th Percentile (%)', '99th Percentile (%)',
#        'Volume ($Billions)']]
# df_sofr['Date'] = pd.to_datetime(df_sofr['Effective Date'], format="%m/%d/%Y")
# df_sofr.index = df_sofr['Date']
# df_sofr.drop(['Effective Date', 'Date'],axis=1, inplace=True)

## Part 1: IRS

In [21]:
usbond_df = pd.read_excel('data/usbond_data.xlsx')
df = pd.read_excel('data/bondtest_data.xlsx')

EURIBOR_3months = 0.0398700
EURIBOR_6months = 0.0405500
EURIBOR_12months = 0.0402800

initial_rates = [EURIBOR_3months, EURIBOR_6months, EURIBOR_12months]

df


,Bond Principal,Time to Maturity,Coupon per year,Bond price
0,100,0.25,0,97.5
1,100,0.50,0,94.9
2,100,1.00,0,90.0
3,100,1.50,8,96.0
4,100,2.00,12,101.6


In [38]:
payments_per_year = 0.5  # (half-yearly)

def bootstrap_yield_curve(test_df):
    zero_rates = {}

    for index, bond in test_df.iterrows():
        print(index)
        TTM = bond['Time to Maturity']
        price = bond['Bond price']
        principle = bond['Bond Principal']
        coupon_rate = bond['Coupon per year'] * payments_per_year

        if coupon_rate == 0:
            # Zero-coupon bond
            zero_rate = -np.log(price / principle) * 1 / TTM
        else:
            # Coupon-bearing bond
            total_discounted_cash_flow = []
            for t in np.arange(0.5, TTM + 0.5, 0.5):
                cash_flow = coupon_rate

                if t in zero_rates:
                    discount_factor = np.exp(-zero_rates[t] * t)
                    total_discounted_cash_flow += cash_flow * discount_factor

            coupon_payment = total_discounted_cash_flow

            zero_rate = -np.log((price + coupon_payment) / (principle + coupon_rate)) * (1 / TTM)

        # Store zero rate using a unique identifier (e.g., index)
        zero_rates[TTM] = zero_rate

    return zero_rates

result = bootstrap_yield_curve(df)
print(result)


0
1
2
3
4
{0.25: 0.10127123193715959, 0.5: 0.10469296074441824, 1.0: 0.10536051565782628, 1.5: 0.0038830816899018236, 2.0: -0.05640801789122627}


In [34]:
EURIBOR_3months = 0.0398700
EURIBOR_6months = 0.0405500
EURIBOR_12months = 0.0402800
initial_rates = [EURIBOR_3months, EURIBOR_6months, EURIBOR_12months]

#df['Bond Principal']
print(df)


#for i in range(len(df)):

#print(df['Bond Principal'][0])


coupon_payments_per_year = 2
dt = df['Time to Maturity']
price = df['Bond price']
TTM = df['Time to Maturity']
principle = df['Bond Principal']


dt = 1/payments_per_year
total_payments = TTM/dt

zero_rates = []
for i in range(len(df)):
    TTM = df['Time to Maturity'][i]
    price = df['Bond price'][i]
    principle = df['Bond Principal'][i]

    coupon_rate = df['Coupon per year'][i] / payments_per_year
    if coupon_rate == 0:
        zero_rate = -np.log(price / principle) * (1 / TTM)
    else:
        disc_coup = []
        for j in range(i-1):
            print(j)
            disc_coup.append(-coupon_rate * np.exp(-zero_rates[j] * dt * (j + 1)))

        
        coupon_payment = np.sum(disc_coup)
        zero_rate = -np.log((price + coupon_payment) / (principle+coupon_rate)) * (1 / TTM)

    zero_rates.append(zero_rate)

    # print(disc_coup, 'dis')
print(zero_rates)

   Bond Principal  Time to Maturity  Coupon per year  Bond price
0             100              0.25                0        97.5
1             100              0.50                0        94.9
2             100              1.00                0        90.0
3             100              1.50                8        96.0
4             100              2.00               12       101.6
0
1
0
1
2
[0.10127123193715959, 0.10469296074441824, 0.10536051565782628, 0.3141831621134138, 0.42069945953501603]
